# Загрузка модулей:

In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

import sys
sys.path.append('./src')  # Для Docker из ./ (./run.sh)
sys.path.append('../')    # Для Docker из dl_utils (dl_utils/docker/run.sh)

In [2]:
from gdinoal import GDino
from ipy_utils import show_video, path2link
from cvat import dir2unlabeled_tasks, tasks_auto_annottation, tasks2preview, subtask2xml
from cvat_srv import CVATSRV, tqdm

2025-11-09 09:24:21.317064: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-09 09:24:21.344627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762680261.361001   29971 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762680261.366295   29971 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762680261.400986   29971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Индексация файлов для предразметки:

In [3]:
path = 'path/to/dir'               # Путь до папки
path = 'path/to/file'              # Путь до файла
path = ['list', 'to', 'files']     # Или список путей до файлов
path = './project'

tasks = dir2unlabeled_tasks(path, desc='Индексация ресурсов')

Индексация ресурсов: 100%|██████████| 2/2 [00:00<00:00, 123.73it/s]


# Загрузка модели и установка параметров предразметки:

In [8]:
def same(x):
    print(len(x))
    return x

gdino = GDino(
    '/home/user/models/groundingdino_swinb_cogcoor.pth',
    box_threshold=0.15,
    text_threshold=0.15,
    prompt2label={
        'UFO': 'ufo',
        'Plane': 'ufo',
        'Bird': 'ufo',
        'Airship': 'ufo',
        'Helicopter': 'ufo',

        'Boat': 'boat',
        'Speedboat': 'boat',
        'Motor boat': 'boat',
        'Sailboat': 'boat',
        'Cruiser': 'boat',
        'Ship': 'boat',
        'Steamboat': 'boat',

        'Object': 'object',
        'Anything': 'object',
        'Stuff': 'object',

        'Person': 'person',
    },
    postprocess_filters=[]
)
undefined_label = 'undefined'

gdino.device

final text_encoder_type: bert-base-uncased


device(type='cuda')

In [38]:
file = './project/videos/7de7cb43a0d811f08be62ea4587eca26_1.jpeg'
img, boxes, logits, labels = gdino._predict(file, gdino.prompt2label, gdino.box_threshold, gdino.text_threshold)

imsize = img.shape[:2]

bboxes = gdino._predict2bboxes(boxes, logits, labels, imsize)

In [41]:
bboxes[0].attribs

{'confidence': 0.5083, 'label': 'person', 'source': 'GroundingDINO'}

In [9]:
from utils import TimeIt

with TimeIt('разметку'):
    annotated_tasks1 = tasks_auto_annottation(tasks, gdino, label=undefined_label)

Авторазметка: 100%|██████████| 2/2 [02:06<00:00, 63.08s/it] 

На разметку потрачено 126.16 секунд.


In [36]:
df1

,track_id,label,frame,true_frame,type,points,occluded,outside,z_order,rotation,attributes,mutable_attributes,group,source,elements
0,None,object,1,1,rectangle,"[5.300708770751953, 1.0314102172851562, 1339.2...",False,False,0,0.0,[],[],0,GroundingDINO,[]
1,None,object,2,2,rectangle,"[1.4956283569335938, 0.6199493408203125, 1343....",False,False,0,0.0,[],[],0,GroundingDINO,[]
2,None,object,3,3,rectangle,"[11.026714324951172, 4.877815246582031, 1327.4...",False,False,0,0.0,[],[],0,GroundingDINO,[]
3,None,person,4,4,rectangle,"[944.4285950660706, 369.2003059387207, 1148.19...",False,False,0,0.0,[],[],0,GroundingDINO,[]
4,None,person,4,4,rectangle,"[280.2476291656494, 384.1483497619629, 452.797...",False,False,0,0.0,[],[],0,GroundingDINO,[]
5,None,object,6,6,rectangle,"[2.3215484619140625, 0.9174728393554688, 1340....",False,False,0,0.0,[],[],0,GroundingDINO,[]
6,None,person,7,7,rectangle,"[644.411151766777, 501.8403797149658, 700.0307...",False,False,0,0.0,[],[],0,GroundingDINO,[]
7,None,object,8,8,rectangle,"[1.6401042938232422, 0.31329345703125, 1343.69...",False,False,0,0.0,[],[],0,GroundingDINO,[]
8,None,ufo,9,9,rectangle,"[21.051189422607422, 1.0862045288085938, 1320....",False,False,0,0.0,[],[],0,GroundingDINO,[]
9,None,ufo,9,9,rectangle,"[20.515302658081055, 0.42551422119140625, 1320...",False,False,0,0.0,[],[],0,GroundingDINO,[]


In [37]:
df2

,track_id,label,frame,true_frame,type,points,occluded,outside,z_order,rotation,attributes,mutable_attributes,group,source,elements
0,None,object,1,1,rectangle,"[5.300708770751953, 1.0314102172851562, 1339.2...",False,False,0,0.0,[],[],0,GroundingDINO,[]
1,None,object,2,2,rectangle,"[1.4956283569335938, 0.6199493408203125, 1343....",False,False,0,0.0,[],[],0,GroundingDINO,[]
2,None,object,3,3,rectangle,"[11.026714324951172, 4.877815246582031, 1327.4...",False,False,0,0.0,[],[],0,GroundingDINO,[]
3,None,person,4,4,rectangle,"[944.4285950660706, 369.2003059387207, 1148.19...",False,False,0,0.0,[],[],0,GroundingDINO,[]
4,None,person,4,4,rectangle,"[280.2476291656494, 384.1483497619629, 452.797...",False,False,0,0.0,[],[],0,GroundingDINO,[]
5,None,object,6,6,rectangle,"[2.3215484619140625, 0.9174728393554688, 1340....",False,False,0,0.0,[],[],0,GroundingDINO,[]
6,None,person,7,7,rectangle,"[644.411151766777, 501.8403797149658, 700.0307...",False,False,0,0.0,[],[],0,GroundingDINO,[]
7,None,object,8,8,rectangle,"[1.6401042938232422, 0.31329345703125, 1343.69...",False,False,0,0.0,[],[],0,GroundingDINO,[]
8,None,ufo,9,9,rectangle,"[21.051189422607422, 1.0862045288085938, 1320....",False,False,0,0.0,[],[],0,GroundingDINO,[]
9,None,ufo,9,9,rectangle,"[20.515302658081055, 0.42551422119140625, 1320...",False,False,0,0.0,[],[],0,GroundingDINO,[]


In [35]:
import numpy as np

df1 = annotated_tasks[1][0][0]
df2 = annotated_tasks1[1][0][0]

assert df1.shape == df2.shape
for i in tqdm(range(df1.shape[0])):
    for j in range(df1.shape[1]):
        v1 = df1.iloc[i, j]
        v2 = df2.iloc[i, j]

        diff = v1 != v2
        if hasattr(diff, 'any'):
            diff = diff.any()
        try:
            if diff:
                print(v1, v2)
        except Exception as e:
            print(v1, v2)
            raise e

IndexError: list index out of range

In [10]:
from utils import obj_diff

print(obj_diff(annotated_tasks1, annotated_tasks))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# Сборка превью:

In [6]:
with TimeIt('сборку превью'):
    out = show_video(tasks2preview(annotated_tasks[::-1], fps=30), size=320)
out

На сборку превью потрачено 651.32 секунд.


In [7]:
1=2

SyntaxError: cannot assign to literal here. Maybe you meant '==' instead of '='? (3293032332.py, line 1)

# Экспорт в CVAT:

In [ ]:
# Соединение с сервером:
cvat_srv = CVATSRV()
print('Соединение с', str(cvat_srv), 'установлено.')

In [ ]:
# Определяем проект для сохранения задач:

# Имя проекта в CVAT:
proj_name = 'test_proj'

labels = set(dino.prompt2label.values()) | {undefined_label}

# Если проект с зажданным именем существует:
if proj_name in cvat_srv.keys():

    # Берём его:
    proj = cvat_srv[proj_name]

    # Убеждаемся, что все необходимые метки в проекте уже есть:
    proj_labels = {label['name'] for label in proj.labels()}
    diff_labels = labels - proj_labels
    if diff_labels:
        raise ValueError(f'В проекте нет следующих меток: {diff_labels}')

    print('Используется имеющийся проект.')

else:
    proj = cvat_srv.new(proj_name, labels=labels)
    print('Создан новый проект.')

In [ ]:
# Сама выгрузка:

xml_file = 'annotations.xml'
for task in tqdm(annotated_tasks, desc='Выгрузка размеченных задач'):

    assert len(task) == 1

    df, file, true_frames = task[0]

    subtask2xml((df, file, true_frames), xml_file)

    if isinstance(file, str):
        name = os.path.basename(os.path.splitext(file)[0])
    else:
        name = os.path.basename(os.path.dirname(file[0]))

    if name in proj.keys():
        raise ValueError(f'В проекте уже есть задача с именем "{name}"!')

    proj.new(name, file, annotation_file=xml_file)